In [ ]:
# ============================================================================
# CELL 1: Set up the Jupyter environment
# ============================================================================

import sys
import os
from pathlib import Path
import torch

print(" AML Semantic Correspondence - Training-Free Baseline\n")
PROJECT_ROOT = str(Path.home() / "AML")
LOCAL_REPO_NAME = str(Path.home() / "AML_SemanticCorrespondence")
DATA_DIR = f'{PROJECT_ROOT}/dataset' 
CHECKPOINT_DIR = f'{PROJECT_ROOT}/checkpoints'
RESULTS_DIR = f'{PROJECT_ROOT}/results'

# Ensure these directories exist (they will be created inside MyDrive/AML)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Add the local repository directory to the path
sys.path.insert(0, LOCAL_REPO_NAME)

# Check GPU availability
print(f"\n  GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
if torch.cuda.is_available():
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("\n Setup complete!\n")

In [ ]:
# ============================================================================
# CELL 2: Set evaluation parameters
# ============================================================================
backbone_choice = 'dinov2'  # 'dinov2', 'dinov3', 'sam'
finetune_choice = False     # True, False
soft_argmax_choice = False  # True, False
dataset_choice = 'spair'   # 'spair', 'pfwillow'
IMG_SIZE = 512 if backbone_choice != 'dinov2' else 518

In [ ]:
# ============================================================================
# CELL 3: Install SAM dependency
# ============================================================================
if 'sam' in backbone_choice:
    print(f" Installing SAM dependency for {backbone_choice}...")
    !pip install -q git+https://github.com/facebookresearch/segment-anything.git
    print(" SAM dependency installed.\n")
else:
    print(" SAM not selected — skipping.\n")


In [ ]:
# ============================================================================
# CELL 4: Install dependencies
# ============================================================================

print(" Installing dependencies...\n")

# Install the repository requirements.txt
!pip install -q -r {LOCAL_REPO_NAME}/requirements.txt

print(" Dependencies installed!\n")

In [ ]:
# ============================================================================
# CELL 5: Load the dataset
# ============================================================================

from dataset import SPairDataset, PFWillowDataset
from torch.utils.data import DataLoader
from pathlib import Path
DATASET_ROOT = f'{DATA_DIR}/Spair-71k' if dataset_choice == 'spair' else f'{DATA_DIR}/PF-WILLOW'

if not Path(DATASET_ROOT).exists():
    raise FileNotFoundError(f"Dataset not found: {DATASET_ROOT}")

if dataset_choice == 'spair':
    # Load dataset
    test_dataset = SPairDataset(
        root=DATASET_ROOT,
        split='test',
        size='large',
        long_side=IMG_SIZE,
        normalize=True,
        load_segmentation=False
    )

    # Create data loader
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=2,
        pin_memory=torch.cuda.is_available()
    )
else:
    # Load dataset
    test_dataset = PFWillowDataset(
        root=DATASET_ROOT,
        long_side=IMG_SIZE,
        normalize=True
    )

    # Create data loader
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=2,
        collate_fn=PFWillowDataset.collate_fn
    )

In [ ]:
# ============================================================================
# CELL 6: Set up evaluation (models and evaluator)
# ============================================================================

from models import UnifiedBackbone, CorrespondenceMatcher, UnifiedEvaluator
import json
from pathlib import Path

print(f"\n{'='*70}")
print("EVALUATION CONFIGURATION (from Cell 1b)")
print('='*70)
print(f"  Backbone: {backbone_choice}")
print(f"  Finetune: {finetune_choice}")
print(f"  Soft Argmax: {soft_argmax_choice}")
print(f"  Dataset: {dataset_choice}")
print('='*70)

# Initialize the backbone
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nDevice: {device}\n")

backbone = UnifiedBackbone(
    backbone_choice=backbone_choice,
    finetune_choice=finetune_choice,
    checkpoint_dir=CHECKPOINT_DIR,
    device=device
)

# Initialize the matcher
matcher = CorrespondenceMatcher(
    backbone=backbone,
    use_soft_argmax=soft_argmax_choice
)

# Initialize the evaluator
evaluator = UnifiedEvaluator(
    dataloader=test_loader,
    device=device,
    thresholds=[0.05, 0.10, 0.15, 0.20]
)

print("\n Evaluation setup complete!")


In [ ]:
# ============================================================================
# CELL 7: Evaluate backbone
# ============================================================================

# Configuration for the evaluation run
NUM_SAMPLES = None  # None = evaluate all, or set to e.g. 100 for testing

print(f"Evaluating {backbone_choice} on SPair-71k test set")
print(f"Samples: {NUM_SAMPLES if NUM_SAMPLES else 'ALL'}\n")

# Run evaluation
results = evaluator.evaluate(
    matcher=matcher,
    backbone_name=backbone.config.name,
    num_samples=NUM_SAMPLES,
    show_progress=True
)

# Save results
output_filename = backbone_choice
if dataset_choice == 'spair':
    output_filename += '_spair'
else:
    output_filename += '_pfwillow'
if finetune_choice:
    output_filename += '_finetuned'
else:
    output_filename += '_baseline'
if soft_argmax_choice:
    output_filename += '_softargmax'
else:
    output_filename += '_hardargmax'
output_filename += '_results.json'
output_file = Path(RESULTS_DIR) / output_filename

with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n Results saved: {output_file}")
